In [1]:
import pandas  as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import random
np.random.seed(30)
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from collections import Counter
import keras
from keras.layers import Conv2D,Dense,Flatten
import keras.backend as K
from keras.callbacks import ModelCheckpoint
from keras.layers import Input
from keras.models import Model
from sklearn.model_selection import KFold


/home/rithwik/anaconda/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
days = pd.HDFStore('../Combined Datasets/daily.h5','r')
hours = pd.HDFStore('../Combined Datasets/hourly.h5','r')

In [3]:
train_prop = .65
v_prop = .2
test_prop = .15
keys = days.keys()

const = 1000.0

In [4]:
def return_data(keys,n_months):
    

    appliance1 = 'use'
    appliance2 = 'air1'
    
    

    
    
    X = np.array([days[i][appliance1].values for i in keys if (appliance1 in days[i].columns and appliance2 in days[i].columns )])[:,:28*n_months]/const
    y = np.array([hours[i][appliance2].values for i in keys if (appliance1 in hours[i].columns and appliance2 in hours[i].columns )])[:,:(28*24*n_months)]/const
    
    
    lst = [86,82,82,73.9,64.9,61,72,77,84,87.1,84.9,84.9,78.1,73,64.9,73,66.9,78.1,78.1,75,84,87.1,87.1,88,88,86,96.1,93,84,75,77,82.9,93.9,91,91.9,87.1,88,84,86,91,93,82,66.9,75.9,84,86,86,84.9,86,89.1,89.1,87.1,84,86,87.1,75,73,84,89.1,93.9,93,88,91,90,91,91,91.9,91.9,90,86,91.9,93.9,91.9,89.1,87.1,91,91.9,91.9,91.9,91,91,91.9,90,88 ]
    temp_matrix = lst * len(X)
    temp_matrix = np.array(temp_matrix).reshape((len(X)*n_months,4,7))
    
    
    X = X.reshape((len(X)*n_months,4,7))
    
    
    
    
    X = np.expand_dims(X,axis=3)    
    temp_matrix = np.expand_dims(temp_matrix,axis=3)
    
    X = np.concatenate((X,temp_matrix),axis=3)
    #print (X.shape)
    
    
    
    
    y = y.reshape((len(y)*n_months,28*24))
    
    
    
    return X,y


def mae(truth,pred):
    return np.mean(const*np.abs((truth-pred)))

    
    
    

In [5]:
def network():
    
    input_layer = Input(shape=(4,7,2),name="Monthly_Matrix")
    input_layer_2 = Input(shape=(1,),name="Cooling_Degree_Day_Monthly")
    

    op1 = Conv2D(10,kernel_size=(4,1), strides=(1,1),name="Filter_4X1")(input_layer)
    
    op1 = Flatten()(op1)
    
    
    
    op2 = Conv2D(10,kernel_size=(1,7), strides=(1,1),name="Filter_1X7")(input_layer)
    
    op2 = Flatten()(op2)
    
    
    op3 = Conv2D(10,kernel_size=(1,2),strides=(1,1),name="Filter_1X2")(input_layer)
    
    op3 = Flatten()(op3)
    
    op4 = Conv2D(10,kernel_size=(2,1),strides=(1,1),name="Filter_2X1")(input_layer)
    
    op4 = Flatten()(op4)
    
    #op5 = Conv2D(20,kernel_size=(3,3),strides=(1,1),padding='same')(input_layer)
    
    #op5 = Conv2D(20,3,strides=(1,1),padding='same')(input_layer)
    
    #op5 = Flatten()(op5)
    
    #op5 =  keras.layers.Dropout(.2)(op5)
    
    #op5  = Dense(168*4)(op5)
    
    
    
    
    
    
    
    
    
    merge_layer = keras.layers.concatenate([op1, op2, op3,op4],name="Concatenation_1")
    

    
    merge_layer =  keras.layers.Dropout(.1)(merge_layer)
    
    
    
    merge_layer = keras.layers.concatenate(([merge_layer,input_layer_2]),name="Concatenation_2")
    
    final_layer = Dense(168*4,name="Output_Dense",activation='relu')(merge_layer)
    
    
    #final_layer = keras.layers.LeakyReLU(alpha=0.01)(final_layer)
    
    #predictions = Dense(1, activation='sigmoid')(merge_layer)

    
    
    
    model = Model(inputs=[input_layer,input_layer_2], outputs=final_layer)

    
    return model
    
    

In [6]:
def empty_dir(dir_name):
    for i in os.listdir(dir_name):
        os.remove(dir_name+"/"+i)

In [7]:
def fit_model(train_x,train_y,test_x,test_y,v_x,v_y):
    
    model = network()
    n_epochs = 2000
    sgd = keras.optimizers.SGD(lr=0.001)
    rmsprop  = keras.optimizers.RMSprop(lr=.001)
    adagrad = keras.optimizers.adagrad(lr=.001)
    model.compile(optimizer=sgd, loss='mean_squared_error')
    
    dir_name = 'temp'
    empty_dir(dir_name)
    
    print("Started Training!")
    checkpoint = ModelCheckpoint(dir_name+'/single-model-{epoch:03d}.h5', verbose=0, monitor='val_loss',save_best_only=True, mode='auto')  
    model.fit([train_x,train_temp], train_y, epochs=n_epochs, validation_data=[[v_x,v_temp],v_y], callbacks=[checkpoint], verbose=0)

    all_files = os.listdir(dir_name)
    all_files.sort()
    weights_file = all_files[-1]
    
    
    model.load_weights(dir_name+"/"+weights_file)
    
    
    pred_train = model.predict([train_x,train_temp])
    pred_v = model.predict([v_x,v_temp])
    pred_test = model.predict([test_x,test_temp])
    
    train_cost = mae(train_y,pred_train)
    v_cost = mae(v_y,pred_v)
    test_cost = mae(test_y,pred_test)
    
    return train_cost,v_cost,test_cost
    
    
    
    
    
    
    

In [8]:
n_experiments = 10
results_arr = [] 
for exp in range(n_experiments):
    
    keys = days.keys()

    np.random.seed(exp)
    
    np.random.shuffle(keys)
    
    train_homes = keys[:int(len(keys)*train_prop)]
    test_homes = keys[int(len(keys)*train_prop):int(-len(keys)*v_prop)]
    val_homes = keys[int(-len(keys)*v_prop):]
    
    train_x,train_y = return_data(train_homes,3)
    
    test_x,test_y = return_data(test_homes,3)
    
    v_x,v_y  = return_data(val_homes,3)
    
    
    
    vals = [16,18,21]
    
    
    train_temp = np.array(vals*(len(train_x)/3))
    
    v_temp = np.array(vals*(len(v_x)/3))
    
    test_temp = np.array(vals*(len(test_x)/3))
    
    
    
    
    results = fit_model(train_x,train_y,test_x,test_y,v_x,v_y)
    
    print (results)
    
    results_arr.append(results)
    
    

Started Training!
(220.59448, 201.93166, 197.60733)
Started Training!
(198.96013, 210.78577, 274.86588)
Started Training!
(207.07861, 217.63968, 225.57394)
Started Training!
(218.69586, 202.70543, 197.99092)
Started Training!
(190.34816, 168.80612, 334.5787)
Started Training!
(190.09767, 279.67142, 232.14886)
Started Training!
(203.57504, 234.57117, 219.26984)
Started Training!
(226.8818, 145.30919, 244.61682)
Started Training!
(208.37859, 168.45158, 301.9334)
Started Training!
(214.95334, 174.37096, 238.31413)


In [9]:
results_arr = np.array(results_arr)

In [10]:
np.mean(results_arr,axis=0)

array([207.95638, 200.42429, 246.68999], dtype=float32)

In [4]:
days['/1037']

,use,air1,clotheswasher1,dishwasher1,drye1,furnace1,kitchenapp1,microwave1,range1,refrigerator1,waterheater1
date,,,,,,,,,,,
2014-04-01,12021.183594,4958.399902,0.000000,0.000000,0.333333,1631.566650,219.266678,131.166672,373.700012,900.933350,1150.633301
2014-04-02,18396.300781,6327.033203,281.016663,0.000000,1650.233398,2019.466675,219.316666,327.566650,535.983337,665.416687,3083.216797
2014-04-03,20768.066406,6267.133301,148.916672,777.599976,5003.066895,2007.099976,111.199997,44.016666,955.500000,756.950012,1915.150024
2014-04-04,11309.250000,4962.116699,0.000000,0.000000,0.000000,1638.916626,114.033333,59.049999,814.366699,866.866638,604.033325
2014-04-05,3877.333252,13.883333,0.000000,0.000000,0.050000,4.983334,0.000000,37.966667,328.216675,637.766663,749.033325
2014-04-06,6844.666504,4.500000,0.000000,0.000000,0.050000,1.533333,350.383331,124.516663,1320.250000,611.983337,1915.566650
2014-04-07,9394.599609,0.000000,0.000000,806.900024,0.100000,2163.883301,118.416672,57.866669,1072.916626,633.316650,2162.783203
2014-04-08,7555.416504,0.000000,0.000000,0.000000,0.000000,1725.566650,115.966667,117.866669,543.766663,755.316650,1987.483276
2014-04-09,6600.416504,0.000000,0.000000,0.000000,0.100000,1049.849976,113.883339,350.049988,790.583313,641.299988,1517.933350
